In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/Colab\ Notebooks/esun_tbrain/dog_sound

Mounted at /gdrive
/gdrive/MyDrive/Colab Notebooks/esun_tbrain/dog_sound


In [ ]:
%cd /gdrive/MyDrive/Colab\ Notebooks/esun_tbrain/dog_sound

/gdrive/MyDrive/Colab Notebooks/esun_tbrain/dog_sound


In [ ]:
!mkdir -p /content/data/ && unzip data/train.zip -d /content/data/ && unzip data/public_test.zip -d /content/data/ && unzip data/private_test.zip -d /content/data/

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/private_test/private_12035.wav  
  inflating: /content/data/private_test/private_05549.wav  
  inflating: /content/data/private_test/private_06040.wav  
  inflating: /content/data/private_test/private_09373.wav  
  inflating: /content/data/private_test/private_16253.wav  
  inflating: /content/data/private_test/private_19160.wav  
  inflating: /content/data/private_test/private_00431.wav  
  inflating: /content/data/private_test/private_10622.wav  
  inflating: /content/data/private_test/private_07438.wav  
  inflating: /content/data/private_test/private_10144.wav  
  inflating: /content/data/private_test/private_00357.wav  
  inflating: /content/data/private_test/private_18518.wav  
  inflating: /content/data/private_test/private_16535.wav  
  inflating: /content/data/private_test/private_19606.wav  
  inflating: /content/data/private_test/private_01049.wav  
  inflating: /content/data/private_test/private_0

In [ ]:
# !wget https://zenodo.org/record/3987831/files/Cnn14_DecisionLevelAtt_mAP%3D0.425.pth?download=1 -O /content/Cnn14_DecisionLevelAtt_mAP0.425.pth
!wget https://zenodo.org/record/3987831/files/Cnn14_mAP%3D0.431.pth?download=1 -O /content/Cnn14_mAP_0.431.pth

--2021-06-10 01:00:03--  https://zenodo.org/record/3987831/files/Cnn14_mAP%3D0.431.pth?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 327428481 (312M) [application/octet-stream]
Saving to: ‘/content/Cnn14_mAP_0.431.pth’

/content/Cnn14_mAP_ 100%[===================>] 312.26M  18.9MB/s    in 18s     

2021-06-10 01:00:23 (17.0 MB/s) - ‘/content/Cnn14_mAP_0.431.pth’ saved [327428481/327428481]



In [ ]:
!pip install -U catalyst
!pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git
!pip install audiomentations

     |████████████████████████████████| 522kB 7.2MB/s 
     |████████████████████████████████| 645kB 17.2MB/s 
     |████████████████████████████████| 122kB 33.5MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/ildoonet/pytorch-gradual-warmup-lr.git to /tmp/pip-req-build-sh8dnkni
  Running command git clone -q https://github.com/ildoonet/pytorch-gradual-warmup-lr.git /tmp/pip-req-build-sh8dnkni
  Created wheel for warmup-scheduler: filename=warmup_scheduler-0.3.2-cp37-none-any.whl size=3880 sha256=6d58816c766201609f4e2bb1acab4134edeba0082dee4ec1b7c5d3774b6567c2
  Stored in directory: /tmp/pip-ephem-wheel-cache-k1f9uqu3/wheels/b7/24/83/d30234cc013cff538805b14df916e79091f7cf9ee2c5bf3a64
Successfully built warmup-scheduler


In [ ]:
import cv2
import audioread
import logging
import os
import random
import time
import warnings

from tqdm import tqdm

import librosa
import librosa.display as display
import numpy as np
import pandas as pd
import soundfile as sf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

from contextlib import contextmanager
from IPython.display import Audio
from pathlib import Path
from typing import Optional, List

from catalyst.dl import SupervisedRunner, CallbackOrder, Callback, CheckpointCallback  # ,State 
from fastprogress import progress_bar
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, average_precision_score, roc_auc_score

from warmup_scheduler import GradualWarmupScheduler


In [ ]:
from catalyst import dl

In [ ]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
    
def get_logger(out_file=None):
    logger = logging.getLogger()
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    logger.handlers = []
    logger.setLevel(logging.INFO)

    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
    handler.setLevel(logging.INFO)
    logger.addHandler(handler)

    if out_file is not None:
        fh = logging.FileHandler(out_file)
        fh.setFormatter(formatter)
        fh.setLevel(logging.INFO)
        logger.addHandler(fh)
    logger.info("logger set up")
    return logger
    
    
@contextmanager
def timer(name: str, logger: Optional[logging.Logger] = None):
    t0 = time.time()
    msg = f"[{name}] start"
    if logger is None:
        print(msg)
    else:
        logger.info(msg)
    yield

    msg = f"[{name}] done in {time.time() - t0:.2f} s"
    if logger is None:
        print(msg)
    else:
        logger.info(msg)
    
    
set_seed(0)

In [ ]:
INPUT_ROOT = "/content/" 
RAW_DATA = INPUT_ROOT + "data/"
TRAIN_AUDIO_DIR = RAW_DATA + "train/"
# TRAIN_RESAMPLED_AUDIO_DIRS = [
#   INPUT_ROOT / "birdsong-resampled-train-audio-{:0>2}".format(i)  for i in range(5)
# ]
TEST_AUDIO_DIR = RAW_DATA + "public_test/"
# TEST2_AUDIO_DIR = RAW_DATA + "private_test/"

!mv /content/data/private_test/*.wav /content/data/public_test

In [ ]:
import pandas as pd

train = pd.read_csv(RAW_DATA + "meta_train.csv")
train['Filename'] = train['Filename'] + '.wav'
print(train.head())
test = pd.read_csv("/gdrive/My Drive/Colab Notebooks/esun_tbrain/dog_sound/data/sample_submission.csv")
test['Filename'] = test['Filename'] + '.wav'
print(test.head())

          Filename  Label   Remark
0  train_00001.wav      0  Barking
1  train_00002.wav      0  Barking
2  train_00003.wav      0  Barking
3  train_00004.wav      0  Barking
4  train_00005.wav      0  Barking
           Filename  Barking  Howling  Crying  COSmoke  GlassBreaking  Other
0  public_00001.wav   0.1666   0.1666  0.1666   0.1666         0.1666  0.167
1  public_00002.wav   0.1666   0.1666  0.1666   0.1666         0.1666  0.167
2  public_00003.wav   0.1666   0.1666  0.1666   0.1666         0.1666  0.167
3  public_00004.wav   0.1666   0.1666  0.1666   0.1666         0.1666  0.167
4  public_00005.wav   0.1666   0.1666  0.1666   0.1666         0.1666  0.167


### torchlibrosa


In PANNs, `torchlibrosa`, a PyTorch based implementation are used to replace some of the `librosa`'s functions. Here I use some functions of `torchlibrosa`.

Ref: https://github.com/qiuqiangkong/torchlibrosa

In [ ]:
class DFTBase(nn.Module):
    def __init__(self):
        """Base class for DFT and IDFT matrix"""
        super(DFTBase, self).__init__()

    def dft_matrix(self, n):
        (x, y) = np.meshgrid(np.arange(n), np.arange(n))
        omega = np.exp(-2 * np.pi * 1j / n)
        W = np.power(omega, x * y)
        return W

    def idft_matrix(self, n):
        (x, y) = np.meshgrid(np.arange(n), np.arange(n))
        omega = np.exp(2 * np.pi * 1j / n)
        W = np.power(omega, x * y)
        return W
    
    
class STFT(DFTBase):
    def __init__(self, n_fft=2048, hop_length=None, win_length=None, 
        window='hann', center=True, pad_mode='reflect', freeze_parameters=True):
        """Implementation of STFT with Conv1d. The function has the same output 
        of librosa.core.stft
        """
        super(STFT, self).__init__()

        assert pad_mode in ['constant', 'reflect']

        self.n_fft = n_fft
        self.center = center
        self.pad_mode = pad_mode

        # By default, use the entire frame
        if win_length is None:
            win_length = n_fft

        # Set the default hop, if it's not already specified
        if hop_length is None:
            hop_length = int(win_length // 4)

        fft_window = librosa.filters.get_window(window, win_length, fftbins=True)

        # Pad the window out to n_fft size
        fft_window = librosa.util.pad_center(fft_window, n_fft)

        # DFT & IDFT matrix
        self.W = self.dft_matrix(n_fft)

        out_channels = n_fft // 2 + 1

        self.conv_real = nn.Conv1d(in_channels=1, out_channels=out_channels, 
            kernel_size=n_fft, stride=hop_length, padding=0, dilation=1, 
            groups=1, bias=False)

        self.conv_imag = nn.Conv1d(in_channels=1, out_channels=out_channels, 
            kernel_size=n_fft, stride=hop_length, padding=0, dilation=1, 
            groups=1, bias=False)

        self.conv_real.weight.data = torch.Tensor(
            np.real(self.W[:, 0 : out_channels] * fft_window[:, None]).T)[:, None, :]
        # (n_fft // 2 + 1, 1, n_fft)

        self.conv_imag.weight.data = torch.Tensor(
            np.imag(self.W[:, 0 : out_channels] * fft_window[:, None]).T)[:, None, :]
        # (n_fft // 2 + 1, 1, n_fft)

        if freeze_parameters:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, input):
        """input: (batch_size, data_length)
        Returns:
          real: (batch_size, n_fft // 2 + 1, time_steps)
          imag: (batch_size, n_fft // 2 + 1, time_steps)
        """

        x = input[:, None, :]   # (batch_size, channels_num, data_length)

        if self.center:
            x = F.pad(x, pad=(self.n_fft // 2, self.n_fft // 2), mode=self.pad_mode)

        real = self.conv_real(x)
        imag = self.conv_imag(x)
        # (batch_size, n_fft // 2 + 1, time_steps)

        real = real[:, None, :, :].transpose(2, 3)
        imag = imag[:, None, :, :].transpose(2, 3)
        # (batch_size, 1, time_steps, n_fft // 2 + 1)

        return real, imag
    
    
class Spectrogram(nn.Module):
    def __init__(self, n_fft=2048, hop_length=None, win_length=None, 
        window='hann', center=True, pad_mode='reflect', power=2.0, 
        freeze_parameters=True):
        """Calculate spectrogram using pytorch. The STFT is implemented with 
        Conv1d. The function has the same output of librosa.core.stft
        """
        super(Spectrogram, self).__init__()

        self.power = power

        self.stft = STFT(n_fft=n_fft, hop_length=hop_length, 
            win_length=win_length, window=window, center=center, 
            pad_mode=pad_mode, freeze_parameters=True)

    def forward(self, input):
        """input: (batch_size, 1, time_steps, n_fft // 2 + 1)
        Returns:
          spectrogram: (batch_size, 1, time_steps, n_fft // 2 + 1)
        """

        (real, imag) = self.stft.forward(input)
        # (batch_size, n_fft // 2 + 1, time_steps)

        spectrogram = real ** 2 + imag ** 2

        if self.power == 2.0:
            pass
        else:
            spectrogram = spectrogram ** (power / 2.0)

        return spectrogram

    
class LogmelFilterBank(nn.Module):
    def __init__(self, sr=32000, n_fft=2048, n_mels=64, fmin=50, fmax=14000, is_log=True, 
        ref=1.0, amin=1e-10, top_db=80.0, freeze_parameters=True):
        """Calculate logmel spectrogram using pytorch. The mel filter bank is 
        the pytorch implementation of as librosa.filters.mel 
        """
        super(LogmelFilterBank, self).__init__()

        self.is_log = is_log
        self.ref = ref
        self.amin = amin
        self.top_db = top_db

        self.melW = librosa.filters.mel(sr=sr, n_fft=n_fft, n_mels=n_mels,
            fmin=fmin, fmax=fmax).T
        # (n_fft // 2 + 1, mel_bins)

        self.melW = nn.Parameter(torch.Tensor(self.melW))

        if freeze_parameters:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, input):
        """input: (batch_size, channels, time_steps)
        
        Output: (batch_size, time_steps, mel_bins)
        """

        # Mel spectrogram
        mel_spectrogram = torch.matmul(input, self.melW)

        # Logmel spectrogram
        if self.is_log:
            output = self.power_to_db(mel_spectrogram)
        else:
            output = mel_spectrogram

        return output


    def power_to_db(self, input):
        """Power to db, this function is the pytorch implementation of 
        librosa.core.power_to_lb
        """
        ref_value = self.ref
        log_spec = 10.0 * torch.log10(torch.clamp(input, min=self.amin, max=np.inf))
        log_spec -= 10.0 * np.log10(np.maximum(self.amin, ref_value))

        if self.top_db is not None:
            if self.top_db < 0:
                raise ParameterError('top_db must be non-negative')
            log_spec = torch.clamp(log_spec, min=log_spec.max().item() - self.top_db, max=np.inf)

        return log_spec

In [ ]:
class DropStripes(nn.Module):
    def __init__(self, dim, drop_width, stripes_num):
        """Drop stripes. 
        Args:
          dim: int, dimension along which to drop
          drop_width: int, maximum width of stripes to drop
          stripes_num: int, how many stripes to drop
        """
        super(DropStripes, self).__init__()

        assert dim in [2, 3]    # dim 2: time; dim 3: frequency

        self.dim = dim
        self.drop_width = drop_width
        self.stripes_num = stripes_num

    def forward(self, input):
        """input: (batch_size, channels, time_steps, freq_bins)"""

        assert input.ndimension() == 4

        if self.training is False:
            return input

        else:
            batch_size = input.shape[0]
            total_width = input.shape[self.dim]

            for n in range(batch_size):
                self.transform_slice(input[n], total_width)

            return input


    def transform_slice(self, e, total_width):
        """e: (channels, time_steps, freq_bins)"""

        for _ in range(self.stripes_num):
            distance = torch.randint(low=0, high=self.drop_width, size=(1,))[0]
            bgn = torch.randint(low=0, high=total_width - distance, size=(1,))[0]

            if self.dim == 2:
                e[:, bgn : bgn + distance, :] = 0
            elif self.dim == 3:
                e[:, :, bgn : bgn + distance] = 0


class SpecAugmentation(nn.Module):
    def __init__(self, time_drop_width, time_stripes_num, freq_drop_width, 
        freq_stripes_num):
        """Spec augmetation. 
        [ref] Park, D.S., Chan, W., Zhang, Y., Chiu, C.C., Zoph, B., Cubuk, E.D. 
        and Le, Q.V., 2019. Specaugment: A simple data augmentation method 
        for automatic speech recognition. arXiv preprint arXiv:1904.08779.
        Args:
          time_drop_width: int
          time_stripes_num: int
          freq_drop_width: int
          freq_stripes_num: int
        """

        super(SpecAugmentation, self).__init__()

        self.time_dropper = DropStripes(dim=2, drop_width=time_drop_width, 
            stripes_num=time_stripes_num)

        self.freq_dropper = DropStripes(dim=3, drop_width=freq_drop_width, 
            stripes_num=freq_stripes_num)

    def forward(self, input):
        x = self.time_dropper(input)
        x = self.freq_dropper(x)
        return x

### audioset_tagging_cnn

I also use `Cnn14_DecisionLevelAtt` model from [PANNs models](https://github.com/qiuqiangkong/audioset_tagging_cnn/blob/master/pytorch/models.py), which is a SED model.

### Building blocks

In [ ]:
def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)


def init_bn(bn):
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.0)


def interpolate(x: torch.Tensor, ratio: int):
    """Interpolate data in time domain. This is used to compensate the
    resolution reduction in downsampling of a CNN.

    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output: torch.Tensor, frames_num: int):
    """Pad framewise_output to the same length as input frames. The pad value
    is the same as the value of the last frame.
    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad
    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    pad = framewise_output[:, -1:, :].repeat(
        1, frames_num - framewise_output.shape[1], 1)
    """tensor for padding"""

    output = torch.cat((framewise_output, pad), dim=1)
    """(batch_size, frames_num, classes_num)"""

    return output


class ConvBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()

        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
            bias=False)

        self.conv2 = nn.Conv2d(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
            bias=False)

        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()

    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

    def forward(self, input, pool_size=(2, 2), pool_type='avg'):

        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')

        return x


class AttBlock(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear",
                 temperature=1.0):
        super().__init__()

        self.activation = activation
        self.temperature = temperature
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.bn_att = nn.BatchNorm1d(out_features)
        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)
        init_bn(self.bn_att)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.clamp(self.att(x), -10, 10), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)
        elif self.activation == 'softmax':
            return nn.LogSoftmax(dim=-1)(x)

In [ ]:
def do_mixup(x, mixup_lambda):
    """Mixup x of even indexes (0, 2, 4, ...) with x of odd indexes 
    (1, 3, 5, ...).
    Args:
      x: (batch_size * 2, ...)
      mixup_lambda: (batch_size * 2,)
    Returns:
      out: (batch_size, ...)
    """
    out = (x[0 :: 2].transpose(0, -1) * mixup_lambda[0 :: 2] + \
        x[1 :: 2].transpose(0, -1) * mixup_lambda[1 :: 2]).transpose(0, -1)
    return out


class Mixup(object):
    def __init__(self, mixup_alpha, random_seed=1234):
        """Mixup coefficient generator.
        """
        self.mixup_alpha = mixup_alpha
        self.random_state = np.random.RandomState(random_seed)

    def get_lambda(self, batch_size):
        """Get mixup random coefficients.
        Args:
          batch_size: int
        Returns:
          mixup_lambdas: (batch_size,)
        """
        mixup_lambdas = []
        for n in range(0, batch_size, 2):
            lam = self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
            mixup_lambdas.append(lam)
            mixup_lambdas.append(1. - lam)

        return np.array(mixup_lambdas)

class Cnn14(nn.Module):
    def __init__(self, sample_rate, window_size, hop_size, mel_bins, fmin, 
        fmax, classes_num):
        
        super(Cnn14, self).__init__()

        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        top_db = None

        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(n_fft=window_size, hop_length=hop_size, 
            win_length=window_size, window=window, center=center, pad_mode=pad_mode, 
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(sr=sample_rate, n_fft=window_size, 
            n_mels=mel_bins, fmin=fmin, fmax=fmax, ref=ref, amin=amin, top_db=top_db, 
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, 
            freq_drop_width=8, freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(64)

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock(in_channels=64, out_channels=128)
        self.conv_block3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv_block4 = ConvBlock(in_channels=256, out_channels=512)
        self.conv_block5 = ConvBlock(in_channels=512, out_channels=1024)
        self.conv_block6 = ConvBlock(in_channels=1024, out_channels=2048)

        self.fc1 = nn.Linear(2048, 2048, bias=True)
        self.fc_audioset = nn.Linear(2048, classes_num, bias=True)
        
        self.init_weight()

    def init_weight(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        init_layer(self.fc_audioset)
 
    def forward(self, input, mixup_lambda=None):
        """
        Input: (batch_size, data_length)"""

        x = self.spectrogram_extractor(input)   # (batch_size, 1, time_steps, freq_bins)
        x = self.logmel_extractor(x)    # (batch_size, 1, time_steps, mel_bins)

        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        
        if self.training:
            x = self.spec_augmenter(x)

        # Mixup on spectrogram
        if self.training and mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)

        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block2(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block3(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block4(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block5(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block6(x, pool_size=(1, 1), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = torch.mean(x, dim=3)
        
        (x1, _) = torch.max(x, dim=2)
        x2 = torch.mean(x, dim=2)
        x = x1 + x2
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu_(self.fc1(x))
        embedding = F.dropout(x, p=0.5, training=self.training)
        # clipwise_output = torch.sigmoid(self.fc_audioset(x))
        clipwise_output = self.fc_audioset(x)
        
        output_dict = {'clipwise_output': clipwise_output, 'embedding': embedding}

        return output_dict

class Transfer_Cnn14(nn.Module):
    def __init__(self, sample_rate, window_size, hop_size, mel_bins, fmin, 
        fmax, classes_num, freeze_base):
        """Classifier for a new task using pretrained Cnn14 as a sub module.
        """
        super(Transfer_Cnn14, self).__init__()
        audioset_classes_num = 527
        
        self.base = Cnn14(sample_rate, window_size, hop_size, mel_bins, fmin, 
            fmax, audioset_classes_num)

        # Transfer to another task layer
        self.fc_transfer = nn.Linear(2048, classes_num, bias=True)

        if freeze_base:
            # Freeze AudioSet pretrained layers
            for param in self.base.parameters():
                param.requires_grad = False

        self.init_weights()

    def init_weights(self):
        init_layer(self.fc_transfer)

    def load_from_pretrain(self, pretrained_checkpoint_path):
        checkpoint = torch.load(pretrained_checkpoint_path)
        self.base.load_state_dict(checkpoint['model'])

    def forward(self, input, mixup_lambda=None):
        """Input: (batch_size, data_length)
        """
        output_dict = self.base(input, mixup_lambda)
        embedding = output_dict['embedding']

        clipwise_output =  torch.log_softmax(self.fc_transfer(embedding), dim=-1)
        output_dict['clipwise_output'] = clipwise_output
 
        return output_dict

What is good in PANNs models is that they accept raw audio clip as input. Let's put a chunk into the CNN feature extractor of the model above.

In [ ]:
path_train_data = '/content/data/train'
path_test_data = '/content/data/public_test'
path_meta = '/content/data/meta_train.csv'
path_submission = 'data/sample_submission.csv'
CLASSES = 6
SR = 8000  # 8000
sample_duration = 40000
hop_length = 256 # making it 128 in size
fmin = 0
fmax = 4000  # fmax <= sampling_rate
n_mels = 64  # 128
n_fft = 1024 # n_mels * 20
padmode = 'reflect'
res_type = 'kaiser_best'  # 'kaiser_fast', 'kaiser_best

# (n+1)*n_fft-hop=40000

train_ratio = 0.9
val_ratio = 1 - train_ratio

warmup_epo = 20
cosine_epo = 180
n_epochs = warmup_epo + cosine_epo
warmup_factor = 10
batch_size = 128
LR = 5e-4

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

kernel_type = 'starter'  # experiment_name
use_fold = 0  # no use

cuda


In [ ]:
model_config = {
    "sample_rate": SR,
    "window_size": n_fft,
    "hop_size": hop_length,
    "mel_bins": n_mels,
    "fmin": fmin,
    "fmax": fmax,
    "classes_num": CLASSES,
    "freeze_base": False
}

# model = PANNsCNN14Att(**model_config)

### Dataset

In [ ]:
PERIOD = 5

class PANNsDataset(data.Dataset):
    def __init__(
            self,
            file_list: List[List[str]],
            waveform_transforms=None,
            training=True):
        self.file_list = file_list  # list of list: [file_path, code]
        self.waveform_transforms = waveform_transforms
        self.training = training

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx: int):
        if self.training:
            wav_path, code = self.file_list[idx]

            y, sr = sf.read(TRAIN_AUDIO_DIR + wav_path)

            if self.waveform_transforms:
                # y = self.waveform_transforms(y)
                y = self.waveform_transforms(samples=y, sample_rate=8000)
            
            # else:

            len_y = len(y)
            effective_length = sr * PERIOD
            if len_y < effective_length:
                new_y = np.zeros(effective_length, dtype=y.dtype)
                start = np.random.randint(effective_length - len_y)
                new_y[start:start + len_y] = y
                y = new_y.astype(np.float32)
            elif len_y > effective_length:
                start = np.random.randint(len_y - effective_length)
                y = y[start:start + effective_length].astype(np.float32)
            else:
                y = y.astype(np.float32)

            labels = np.zeros(CLASSES, dtype="f")
            labels[code] = 1

            return {"waveform": y, "targets": labels}
        else:
            wav_path = self.file_list[idx]

            y, sr = sf.read(TEST_AUDIO_DIR + wav_path)

            len_y = len(y)
            effective_length = sr * PERIOD
            if len_y < effective_length:
                new_y = np.zeros(effective_length, dtype=y.dtype)
                start = np.random.randint(effective_length - len_y)
                new_y[start:start + len_y] = y
                y = new_y.astype(np.float32)
            elif len_y > effective_length:
                start = np.random.randint(len_y - effective_length)
                y = y[start:start + effective_length].astype(np.float32)
            else:
                y = y.astype(np.float32)

            return {"waveform": y}

### Criterion

In [ ]:
class PANNsLoss(nn.Module):
    def __init__(self):
        super().__init__()

        # self.bce = nn.BCELoss()
        # self.bce = nn.BCEWithLogitsLoss()
        self.bce = nn.NLLLoss()

    def forward(self, input, target):
        input_ = input["clipwise_output"]
        input_ = torch.where(torch.isnan(input_),
                             torch.zeros_like(input_),
                             input_)
        input_ = torch.where(torch.isinf(input_),
                             torch.zeros_like(input_),
                             input_)

        # target = target.float()

        return self.bce(input_, target)

In [ ]:
class CutMixCrossEntropyLoss(nn.Module):
    def __init__(self, size_average=True):
        super().__init__()
        self.size_average = size_average

    def forward(self, input, target):
        if len(target.size()) == 1:
            target = F.one_hot(target, num_classes=input.size(-1))
            target = target.float().cuda()
        return cross_entropy(input, target, self.size_average)


def cross_entropy(input, target, size_average=True):
    """ Cross entropy that accepts soft targets
    Args:
         pred: predictions for neural network
         targets: targets, can be soft
         size_average: if false, sum is returned instead of mean
    Examples::
        input = torch.FloatTensor([[1.1, 2.8, 1.3], [1.1, 2.1, 4.8]])
        input = torch.autograd.Variable(out, requires_grad=True)
        target = torch.FloatTensor([[0.05, 0.9, 0.05], [0.05, 0.05, 0.9]])
        target = torch.autograd.Variable(y1)
        loss = cross_entropy(input, target)
        loss.backward()
    """
    # logsoftmax = torch.nn.LogSoftmax(dim=1)
    # if size_average:
    #     return torch.mean(torch.sum(-target * logsoftmax(input), dim=1))
    # else:
    #     return torch.sum(torch.sum(-target * logsoftmax(input), dim=1))
    
    if size_average:
        return torch.mean(torch.sum(-target * (input), dim=1))
    else:
        return torch.sum(torch.sum(-target * (input), dim=1))

### Callbacks

In [ ]:
# class F1Callback(Callback):
#     def __init__(self,
#                  input_key: str = "targets",
#                  output_key: str = "logits",
#                  model_output_key: str = "clipwise_output",
#                  prefix: str = "f1"):
#         super().__init__(CallbackOrder.Metric)

#         self.input_key = input_key
#         self.output_key = output_key
#         self.model_output_key = model_output_key
#         self.prefix = prefix

#     def on_loader_start(self, state: State):
#         self.prediction: List[np.ndarray] = []
#         self.target: List[np.ndarray] = []

#     def on_batch_end(self, state: State):
#         targ = state.input[self.input_key].detach().cpu().numpy()
#         out = state.output[self.output_key]

#         clipwise_output = out[self.model_output_key].detach().cpu().numpy()

#         self.prediction.append(clipwise_output)
#         self.target.append(targ)

#         y_pred = clipwise_output.argmax(axis=1)
#         y_true = targ.argmax(axis=1)

#         score = f1_score(y_true, y_pred, average="macro")
#         state.batch_metrics[self.prefix] = score

#     def on_loader_end(self, state: State):
#         y_pred = np.concatenate(self.prediction, axis=0).argmax(axis=1)
#         y_true = np.concatenate(self.target, axis=0).argmax(axis=1)
#         score = f1_score(y_true, y_pred, average="macro")
#         state.loader_metrics[self.prefix] = score
#         if state.is_valid_loader:
#             state.epoch_metrics[state.valid_loader + "_epoch_" +
#                                 self.prefix] = score
#         else:
#             state.epoch_metrics["train_epoch_" + self.prefix] = score

# class AUCCallback(Callback):
#     def __init__(self,
#                  input_key: str = "targets",
#                  output_key: str = "logits",
#                  model_output_key: str = "clipwise_output",
#                  prefix: str = "auc"):
#         super().__init__(CallbackOrder.Metric)

#         self.input_key = input_key
#         self.output_key = output_key
#         self.model_output_key = model_output_key
#         self.prefix = prefix

#     def on_loader_start(self, runner):
#         self.prediction: List[np.ndarray] = []
#         self.target: List[np.ndarray] = []

#     def on_batch_end(self, runner):
#         targ = runner.input[self.input_key].detach().cpu().numpy()
#         out = runner.output[self.output_key]

#         clipwise_output = out[self.model_output_key].detach().cpu().numpy()

#         self.prediction.append(clipwise_output)
#         self.target.append(targ)

#         y_pred = clipwise_output.argmax(axis=1)
#         y_true = targ.argmax(axis=1)

#         score = roc_auc_score(y_true, y_pred, average='weighted', multi_class='ovr', labels=list(range(CLASSES)))
#         runner.batch_metrics[self.prefix] = score

#     def on_loader_end(self, runner):
#         y_pred = np.concatenate(self.prediction, axis=0).argmax(axis=1)
#         y_true = np.concatenate(self.target, axis=0).argmax(axis=1)
#         score = roc_auc_score(y_true, y_pred, average='weighted', multi_class='ovr', labels=list(range(CLASSES)))
#         runner.loader_metrics[self.prefix] = score
#         if runner.is_valid_loader:
#             runner.epoch_metrics[runner.valid_loader + "_epoch_" +
#                                 self.prefix] = score
#         else:
#             runner.epoch_metrics["train_epoch_" + self.prefix] = score

### Train

Some code are taken from https://www.kaggle.com/ttahara/training-birdsong-baseline-resnest50-fast .
Thanks @ttahara!

In [ ]:
print(train.shape)

(1200, 3)


In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

train["fold"] = -1
for fold_id, (train_index, val_index) in enumerate(skf.split(train, train['Label'])):
    train.iloc[val_index, -1] = fold_id

In [ ]:
train_file_list = train.query("fold != @use_fold")[["Filename", "Label"]].values.tolist()
val_file_list = train.query("fold == @use_fold")[["Filename", "Label"]].values.tolist()

print("[fold {}] train: {}, val: {}".format(use_fold, len(train_file_list), len(val_file_list)))

[fold 0] train: 960, val: 240


In [ ]:
# Fix Warmup Bug
class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

In [ ]:
from audiomentations import Compose, AddGaussianNoise, Gain, TimeStretch, PitchShift, Shift, TimeMask, AddGaussianSNR

train_transform = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.3),
    # AddGaussianSNR(p=0.3),
    Gain(min_gain_in_db=-12, max_gain_in_db=12, p=0.3),
    # PitchShift(min_semitones=-4, max_semitones=4, p=0.3),
    TimeStretch(min_rate=0.8, max_rate=1.2, p=0.3),
    Shift(min_fraction=-0.5, max_fraction=0.5, rollover=False, p=0.3),
    # TimeMask(min_band_part=0.0, max_band_part=0.2, p=0.5),
    # FrequencyMask(min_frequency_band=0.0, max_frequency_band=0.2, p=0.5)
])

# loaders
loaders = {
    "train": data.DataLoader(PANNsDataset(train_file_list, None), 
                             batch_size=batch_size, 
                             shuffle=True, 
                             num_workers=2, 
                             pin_memory=True, 
                             drop_last=True),
    "valid": data.DataLoader(PANNsDataset(val_file_list, None), 
                             batch_size=batch_size, 
                             shuffle=False,
                             num_workers=2,
                             pin_memory=True,
                             drop_last=False)
}

train_loader = data.DataLoader(PANNsDataset(train_file_list, train_transform), 
                             batch_size=batch_size, 
                             shuffle=True, 
                             num_workers=2, 
                             pin_memory=True, 
                             drop_last=False)
val_loader = data.DataLoader(PANNsDataset(val_file_list, None), 
                             batch_size=batch_size, 
                             shuffle=False,
                             num_workers=2,
                             pin_memory=True,
                             drop_last=False)

# model
# model_config["classes_num"] = 527  # for load weights only 
# model = PANNsCNN14Att(**model_config)
# weights = torch.load(INPUT_ROOT + "Cnn14_DecisionLevelAtt_mAP0.425.pth")
# # Fixed in V3
# model.load_state_dict(weights["model"])
# model.att_block = AttBlock(2048, CLASSES, activation='softmax')
# model.att_block.init_weights()
# model.to(device)

# TODO
model = Transfer_Cnn14(**model_config)
model.load_from_pretrain(INPUT_ROOT + "Cnn14_mAP_0.431.pth")
model.to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=LR)

# Scheduler
# scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs-warmup_epo)
scheduler = GradualWarmupSchedulerV2(optimizer, multiplier=warmup_factor, total_epoch=warmup_epo, after_scheduler=scheduler_cosine)

# Loss
# criterion = PANNsLoss().to(device)
criterion = CutMixCrossEntropyLoss().to(device)

# callbacks
# callbacks = [
#     AUCCallback(input_key="targets", output_key="logits", prefix="auc"),
#     # CheckpointCallback(save_n_best=0)
#     # F1Callback(input_key="targets", output_key="logits", prefix="f1"),
#     # mAPCallback(input_key="targets", output_key="logits", prefix="mAP"),
    
# ]

In [ ]:
def train_epoch(loader, optimizer):
    model.train()
    train_loss = []
    LOGITS = []
    PREDS = []
    TARGETS = []

    bar = tqdm(loader)
    for d in bar:
        data, target = d['waveform'], d['targets']
        optimizer.zero_grad()
        scheduler.step()
        
        data, target = data.to(device), target.to(device)
        
        # mixup_augmenter = Mixup(mixup_alpha=1.)
        # mixup_lambda = mixup_augmenter.get_lambda(batch_size=len(data))
        # mixup_lambda = torch.tensor(mixup_lambda, dtype=torch.float32).to(device)
        # target_mix = do_mixup(target, mixup_lambda)
       
        logits = model(data, mixup_lambda=None)
        # print(logits['framewise_output'].shape)    # shape (batch_size, frame_size, classes)
        # print(logits['clipwise_output'].shape)    # shape (batch_size, classes)
        logits_clip = logits['clipwise_output']

        # loss = criterion(logits, target)
        ### hard cross entropy
        loss = criterion(logits_clip, (target==1).nonzero(as_tuple=True)[1].to(torch.long))
        ### soft cross entropy
        # loss = criterion(logits_clip, target_mix)

        loss.backward()
        optimizer.step()

        loss_np = loss.detach().cpu().numpy()
        train_loss.append(loss_np)
        smooth_loss = sum(train_loss[-100:]) / min(len(train_loss), 100)
        bar.set_description('loss: %.5f, smth: %.5f' % (loss_np, smooth_loss))

        with torch.no_grad():
            # pred = logits_clip.argmax(dim=1).detach()
            logits_org = model(data)
            LOGITS.append(logits_org['clipwise_output'])
            PREDS.append(logits_org['clipwise_output'])

            # to one hot
            # target = F.one_hot(target, num_classes=CLASSES)
            TARGETS.append(target)

    PREDS = torch.cat(PREDS).detach().cpu().numpy()
    TARGETS = torch.cat(TARGETS).cpu().numpy()
    auc = roc_auc_score(TARGETS, PREDS, average='weighted', multi_class='ovr', labels=list(range(CLASSES)))
    return train_loss, auc


def val_epoch(loader, get_output=False):

    model.eval()
    val_loss = []
    LOGITS = []
    PREDS = []
    TARGETS = []

    with torch.no_grad():
        for d in tqdm(loader):
            data, target = d['waveform'], d['targets']
            data, target = data.to(device), target.to(device)
            logits = model(data)
            logits_clip = logits['clipwise_output']

            # loss = criterion(logits, target)
            loss = criterion(logits_clip, (target==1).nonzero(as_tuple=True)[1].to(torch.long))

            pred = logits_clip.argmax(dim=1).detach()
            LOGITS.append(logits_clip)
            PREDS.append(logits_clip)
            
            # to one hot
            # target = F.one_hot(target, num_classes=CLASSES)
            TARGETS.append(target)

            val_loss.append(loss.detach().cpu().numpy())
        val_loss = np.mean(val_loss)

    LOGITS = torch.cat(LOGITS).cpu().numpy()
    PREDS = torch.cat(PREDS).detach().cpu().numpy()
    TARGETS = torch.cat(TARGETS).cpu().numpy()
    auc = roc_auc_score(TARGETS, PREDS, average='weighted', multi_class='ovr', labels=list(range(CLASSES)))

    if get_output:
        return LOGITS
    else:
        return val_loss, auc

auc_max = 0
best_file = f'{kernel_type}_best_fold{use_fold}.pth'
for epoch in range(1, n_epochs+1):
    print(time.ctime(), 'Epoch:', epoch)
    scheduler.step(epoch-1)

    train_loss, tr_auc = train_epoch(train_loader, optimizer)
    val_loss, val_auc = val_epoch(val_loader)

    content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {np.mean(train_loss):.5f}, train auc: {(tr_auc):.5f}, val loss: {np.mean(val_loss):.5f}, val auc: {(val_auc):.5f}'
    print(content)
    with open(f'log_{kernel_type}.txt', 'a') as appender:
        appender.write(content + '\n')

    if val_auc > auc_max:
        print('score2 ({:.6f} --> {:.6f}).  Saving model ...'.format(auc_max, val_auc))
        torch.save(model.state_dict(), best_file)
        auc_max = val_auc

torch.save(model.state_dict(), os.path.join(f'{kernel_type}_final_fold{use_fold}.pth'))

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)

  0%|          | 0/8 [00:00<?, ?it/s]

Thu Jun 10 01:01:46 2021 Epoch: 1



loss: 1.45127, smth: 1.72563: 100%|██████████| 8/8 [00:17<00:00,  2.18s/it]

100%|██████████| 2/2 [00:03<00:00,  1.66s/it]


Thu Jun 10 01:02:07 2021 Epoch 1, lr: 0.0023000, train loss: 1.72563, train auc: 0.61963, val loss: 1.00917, val auc: 0.93760
score2 (0.000000 --> 0.937604).  Saving model ...


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)

  0%|          | 0/8 [00:00<?, ?it/s]

...

Thu Jun 10 01:35:31 2021 Epoch 198, lr: 0.0000095, train loss: 0.12316, train auc: 0.99783, val loss: 0.45003, val auc: 0.98630
Thu Jun 10 01:35:31 2021 Epoch: 199



loss: 0.12513, smth: 0.12602: 100%|██████████| 8/8 [00:09<00:00,  1.21s/it]

100%|██████████| 2/2 [00:00<00:00,  3.77it/s]
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)

  0%|          | 0/8 [00:00<?, ?it/s]

Thu Jun 10 01:35:42 2021 Epoch 199, lr: 0.0000137, train loss: 0.12602, train auc: 0.99794, val loss: 0.43351, val auc: 0.98699
Thu Jun 10 01:35:42 2021 Epoch: 200



loss: 0.08360, smth: 0.14744: 100%|██████████| 8/8 [00:09<00:00,  1.15s/it]

100%|██████████| 2/2 [00:00<00:00,  3.60it/s]


Thu Jun 10 01:35:51 2021 Epoch 200, lr: 0.0000186, train loss: 0.14744, train auc: 0.99842, val loss: 0.45193, val auc: 0.98641


In [ ]:
model = model.to(device)
model.load_state_dict(torch.load(os.path.join(f'{kernel_type}_best_fold{use_fold}.pth')))

<All keys matched successfully>

In [ ]:
# test_file_list = test['Filename'][:10000].values.tolist()
test_file_list = test['Filename'].values.tolist()
test_loader = data.DataLoader(PANNsDataset(test_file_list, None, training=False), 
                             batch_size=16, 
                             shuffle=False,
                             num_workers=2,
                             pin_memory=True,
                             drop_last=False)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
PREDS = []
bar = tqdm(test_loader)
for d in bar:
    data = d['waveform']
    data = data.to(device)
    with torch.no_grad():
        logits = model(data)

        # print(torch.exp(logits['clipwise_output']))
        # print(torch.exp(logits['clipwise_output']).sum(axis=-1))
        # input()
        
        PREDS.append(torch.exp(logits['clipwise_output']))

        # if pred_num != target_num:
        #     plt.imshow(data.squeeze().cpu().numpy())
        #     plt.axis('off')
        #     plt.show()
        #     input('next...')

PREDS = torch.cat(PREDS).detach().cpu().numpy()


100%|██████████| 1875/1875 [01:08<00:00, 27.38it/s]


In [ ]:
# test.iloc[:10000, 1:] = F.softmax(torch.tensor(PREDS), dim=-1).numpy()
# test.iloc[:10000, 1:] = torch.tensor(PREDS).numpy()
test.iloc[:, 1:] = torch.tensor(PREDS).numpy()
test['Filename'] = test['Filename'].str.replace('.wav', '')
test

,Filename,Barking,Howling,Crying,COSmoke,GlassBreaking,Other
0,public_00001,6.144940e-02,5.453053e-01,3.228939e-01,1.126858e-02,7.859051e-03,0.051224
1,public_00002,2.654521e-07,3.957806e-07,1.911393e-07,5.934786e-10,1.516890e-05,0.999984
2,public_00003,9.999226e-01,3.045627e-05,1.043902e-05,7.652030e-08,1.474413e-05,0.000022
3,public_00004,2.964883e-01,1.444015e-01,2.364393e-01,8.931077e-02,8.475937e-02,0.148601
4,public_00005,3.901064e-02,1.406257e-01,7.239787e-01,1.721349e-02,3.508842e-02,0.044083
...,...,...,...,...,...,...,...
29995,private_19996,2.312374e-03,2.550300e-05,2.088708e-03,9.953142e-01,1.351796e-04,0.000124
29996,private_19997,1.420989e-01,1.024586e-01,1.133607e-01,6.746402e-02,2.826460e-01,0.291972
29997,private_19998,8.714300e-10,2.151173e-09,7.284987e-10,1.612620e-13,1.130452e-07,1.000000
29998,private_19999,1.540704e-03,1.168827e-02,9.866976e-01,3.251455e-06,6.343484e-06,0.000064


In [ ]:
test.to_csv('submission.csv', index=False)

In [ ]:
# from catalyst import dl

# warnings.simplefilter("ignore")

# runner = SupervisedRunner(
#     input_key="waveform",
#     output_key="logits",
#     target_key="targets",
#     loss_key="loss"
# )

# runner.train(
#     model=model,
#     criterion=criterion,
#     loaders=loaders,
#     optimizer=optimizer,
#     scheduler=scheduler,
#     num_epochs=10,
#     verbose=True,
#     logdir=None, #f"fold0",
#     callbacks=[
#         dl.AUCCallback(input_key="logits", target_key="targets")  #callbacks,
#     ],
#     valid_loader="valid",
#     valid_metric="loss"
# )

Seems it's learning something.

Now I'll show how this model works in the inference phase. I'll use trained model of this which I trained by myself using the data of this competition in my local environment.

Since [several concerns](https://www.kaggle.com/c/birdsong-recognition/discussion/172356) are expressed about over-sharing of top solutions during competition, and since I do respect those people who have worked hard to improve their scores, I would not make trained weight in common and would not share how I trained this model.

## Prediction with SED model

In [ ]:
model_config = {
    "sample_rate": 32000,
    "window_size": 1024,
    "hop_size": 320,
    "mel_bins": 64,
    "fmin": 50,
    "fmax": 14000,
    "classes_num": 264
}

weights_path = "../input/birdcall-pannsatt-aux-weak/best.pth"

In [ ]:
def get_model(config: dict, weights_path: str):
    model = PANNsCNN14Att(**config)
    checkpoint = torch.load(weights_path)
    model.load_state_dict(checkpoint["model_state_dict"])
    device = torch.device("cuda")
    model.to(device)
    model.eval()
    return model

In [ ]:
def prediction_for_clip(test_df: pd.DataFrame,
                        clip: np.ndarray, 
                        model: PANNsCNN14Att,
                        threshold=0.5):
    PERIOD = 30
    audios = []
    y = clip.astype(np.float32)
    len_y = len(y)
    start = 0
    end = PERIOD * SR
    while True:
        y_batch = y[start:end].astype(np.float32)
        if len(y_batch) != PERIOD * SR:
            y_pad = np.zeros(PERIOD * SR, dtype=np.float32)
            y_pad[:len(y_batch)] = y_batch
            audios.append(y_pad)
            break
        start = end
        end += PERIOD * SR
        audios.append(y_batch)
        
    array = np.asarray(audios)
    tensors = torch.from_numpy(array)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.eval()
    estimated_event_list = []
    global_time = 0.0
    site = test_df["site"].values[0]
    audio_id = test_df["audio_id"].values[0]
    for image in progress_bar(tensors):
        image = image.view(1, image.size(0))
        image = image.to(device)

        with torch.no_grad():
            prediction = model(image)
            framewise_outputs = prediction["framewise_output"].detach(
                ).cpu().numpy()[0]
                
        thresholded = framewise_outputs >= threshold

        for target_idx in range(thresholded.shape[1]):
            if thresholded[:, target_idx].mean() == 0:
                pass
            else:
                detected = np.argwhere(thresholded[:, target_idx]).reshape(-1)
                head_idx = 0
                tail_idx = 0
                while True:
                    if (tail_idx + 1 == len(detected)) or (
                            detected[tail_idx + 1] - 
                            detected[tail_idx] != 1):
                        onset = 0.01 * detected[
                            head_idx] + global_time
                        offset = 0.01 * detected[
                            tail_idx] + global_time
                        onset_idx = detected[head_idx]
                        offset_idx = detected[tail_idx]
                        max_confidence = framewise_outputs[
                            onset_idx:offset_idx, target_idx].max()
                        mean_confidence = framewise_outputs[
                            onset_idx:offset_idx, target_idx].mean()
                        estimated_event = {
                            "site": site,
                            "audio_id": audio_id,
                            "ebird_code": INV_BIRD_CODE[target_idx],
                            "onset": onset,
                            "offset": offset,
                            "max_confidence": max_confidence,
                            "mean_confidence": mean_confidence
                        }
                        estimated_event_list.append(estimated_event)
                        head_idx = tail_idx + 1
                        tail_idx = tail_idx + 1
                        if head_idx >= len(detected):
                            break
                    else:
                        tail_idx += 1
        global_time += PERIOD
        
    prediction_df = pd.DataFrame(estimated_event_list)
    return prediction_df

In [ ]:
def prediction(test_df: pd.DataFrame,
               test_audio: Path,
               model_config: dict,
               weights_path: str,
               threshold=0.5):
    model = get_model(model_config, weights_path)
    unique_audio_id = test_df.audio_id.unique()

    warnings.filterwarnings("ignore")
    prediction_dfs = []
    for audio_id in unique_audio_id:
        with timer(f"Loading {audio_id}"):
            clip, _ = librosa.load(test_audio / (audio_id + ".mp3"),
                                   sr=SR,
                                   mono=True,
                                   res_type="kaiser_fast")
        
        test_df_for_audio_id = test_df.query(
            f"audio_id == '{audio_id}'").reset_index(drop=True)
        with timer(f"Prediction on {audio_id}"):
            prediction_df = prediction_for_clip(test_df_for_audio_id,
                                                clip=clip,
                                                model=model,
                                                threshold=threshold)

        prediction_dfs.append(prediction_df)
    
    prediction_df = pd.concat(prediction_dfs, axis=0, sort=False).reset_index(drop=True)
    return prediction_df

In [ ]:
prediction_df = prediction(test_df=test,
                           test_audio=TEST_AUDIO_DIR,
                           model_config=model_config,
                           weights_path=weights_path,
                           threshold=0.5)

[Loading 41e6fe6504a34bf6846938ba78d13df1] start
[Loading 41e6fe6504a34bf6846938ba78d13df1] done in 2.31 s
[Prediction on 41e6fe6504a34bf6846938ba78d13df1] start


[Prediction on 41e6fe6504a34bf6846938ba78d13df1] done in 0.60 s
[Loading cce64fffafed40f2b2f3d3413ec1c4c2] start
[Loading cce64fffafed40f2b2f3d3413ec1c4c2] done in 0.81 s
[Prediction on cce64fffafed40f2b2f3d3413ec1c4c2] start


[Prediction on cce64fffafed40f2b2f3d3413ec1c4c2] done in 0.07 s
[Loading 99af324c881246949408c0b1ae54271f] start
[Loading 99af324c881246949408c0b1ae54271f] done in 0.82 s
[Prediction on 99af324c881246949408c0b1ae54271f] start


[Prediction on 99af324c881246949408c0b1ae54271f] done in 0.08 s
[Loading 6ab74e177aa149468a39ca10beed6222] start
[Loading 6ab74e177aa149468a39ca10beed6222] done in 0.76 s
[Prediction on 6ab74e177aa149468a39ca10beed6222] start


[Prediction on 6ab74e177aa149468a39ca10beed6222] done in 0.08 s
[Loading b2fd3f01e9284293a1e33f9c811a2ed6] start
[Loading b2fd3f01e9284293a1e33f9c811a2ed6] done in 0.78 s
[Prediction on b2fd3f01e9284293a1e33f9c811a2ed6] start


...

[Prediction on 9cc5d9646f344f1bbb52640a988fe902] done in 0.32 s
[Loading a56e20a518684688a9952add8a9d5213] start
[Loading a56e20a518684688a9952add8a9d5213] done in 0.74 s
[Prediction on a56e20a518684688a9952add8a9d5213] start


[Prediction on a56e20a518684688a9952add8a9d5213] done in 0.08 s
[Loading 96779836288745728306903d54e264dd] start
[Loading 96779836288745728306903d54e264dd] done in 0.59 s
[Prediction on 96779836288745728306903d54e264dd] start


[Prediction on 96779836288745728306903d54e264dd] done in 0.04 s
[Loading f77783ba4c6641bc918b034a18c23e53] start
[Loading f77783ba4c6641bc918b034a18c23e53] done in 0.47 s
[Prediction on f77783ba4c6641bc918b034a18c23e53] start


[Prediction on f77783ba4c6641bc918b034a18c23e53] done in 0.04 s
[Loading 856b194b097441958697c2bcd1f63982] start
[Loading 856b194b097441958697c2bcd1f63982] done in 0.71 s
[Prediction on 856b194b097441958697c2bcd1f63982] start


[Prediction on 856b194b097441958697c2bcd1f63982] done in 0.04 s


In [ ]:
prediction_df

,site,audio_id,ebird_code,onset,offset,max_confidence,mean_confidence
0,site_1,41e6fe6504a34bf6846938ba78d13df1,aldfly,0.96,2.23,0.985395,0.897037
1,site_1,41e6fe6504a34bf6846938ba78d13df1,aldfly,7.04,7.67,0.526611,0.519470
2,site_1,41e6fe6504a34bf6846938ba78d13df1,aldfly,11.20,12.15,0.956318,0.928820
3,site_1,41e6fe6504a34bf6846938ba78d13df1,aldfly,14.40,15.03,0.809643,0.805571
4,site_1,41e6fe6504a34bf6846938ba78d13df1,aldfly,20.16,21.43,0.987058,0.917030
...,...,...,...,...,...,...,...
195,site_3,856b194b097441958697c2bcd1f63982,aldfly,18.24,19.19,0.885321,0.789641
196,site_3,856b194b097441958697c2bcd1f63982,aldfly,19.84,22.07,0.983291,0.913020
197,site_3,856b194b097441958697c2bcd1f63982,aldfly,23.04,23.67,0.669237,0.624711
198,site_3,856b194b097441958697c2bcd1f63982,aldfly,25.92,26.87,0.950051,0.897717


## Postprocess


In [ ]:
labels = {}

for audio_id, sub_df in prediction_df.groupby("audio_id"):
    events = sub_df[["ebird_code", "onset", "offset", "max_confidence", "site"]].values
    n_events = len(events)
    removed_event = []
    # Overlap deletion: this part may not be necessary
    # I deleted this part in other model and found there's no difference on the public LB score.
    for i in range(n_events):
        for j in range(n_events):
            if i == j:
                continue
            if i in removed_event:
                continue
            if j in removed_event:
                continue
            
            event_i = events[i]
            event_j = events[j]
            
            if (event_i[1] - event_j[2] >= 0) or (event_j[1] - event_i[2] >= 0):
                pass
            else:
                later_onset = max(event_i[1], event_j[1])
                sooner_onset = min(event_i[1], event_j[1])
                sooner_offset = min(event_i[2], event_j[2])
                later_offset = max(event_i[2], event_j[2])

                intersection = sooner_offset - later_onset
                union = later_offset - sooner_onset
                
                iou = intersection / union
                if iou > 0.4:
                    if event_i[3] > event_j[3]:
                        removed_event.append(j)
                    else:
                        removed_event.append(i)

    site = events[0][4]
    for i in range(n_events):
        if i in removed_event:
            continue
        event = events[i][0]
        onset = events[i][1]
        offset = events[i][2]
        if site in {"site_1", "site_2"}:
            start_section = int((onset // 5) * 5) + 5
            end_section = int((offset // 5) * 5) + 5
            cur_section = start_section

            row_id = f"{site}_{audio_id}_{start_section}"
            if labels.get(row_id) is not None:
                labels[row_id].add(event)
            else:
                labels[row_id] = set()
                labels[row_id].add(event)

            while cur_section != end_section:
                cur_section += 5
                row_id = f"{site}_{audio_id}_{cur_section}"
                if labels.get(row_id) is not None:
                    labels[row_id].add(event)
                else:
                    labels[row_id] = set()
                    labels[row_id].add(event)
        else:
            row_id = f"{site}_{audio_id}"
            if labels.get(row_id) is not None:
                labels[row_id].add(event)
            else:
                labels[row_id] = set()
                labels[row_id].add(event)

In [ ]:
for key in labels:
    labels[key] = " ".join(sorted(list(labels[key])))
    
    
row_ids = list(labels.keys())
birds = list(labels.values())
post_processed = pd.DataFrame({
    "row_id": row_ids,
    "birds": birds
})
post_processed.head()

,row_id,birds
0,site_2_07ab324c602e4afab65ddbcc746c31b5_5,aldfly
1,site_2_07ab324c602e4afab65ddbcc746c31b5_10,aldfly
2,site_2_07ab324c602e4afab65ddbcc746c31b5_15,aldfly
3,site_2_07ab324c602e4afab65ddbcc746c31b5_25,redcro
4,site_1_41e6fe6504a34bf6846938ba78d13df1_5,aldfly


In [ ]:
all_row_id = test[["row_id"]]
submission = all_row_id.merge(post_processed, on="row_id", how="left")
submission = submission.fillna("nocall")
submission.to_csv("submission.csv", index=False)
submission.head(20)

,row_id,birds
0,site_1_41e6fe6504a34bf6846938ba78d13df1_5,aldfly
1,site_1_41e6fe6504a34bf6846938ba78d13df1_10,aldfly fiespa
2,site_1_41e6fe6504a34bf6846938ba78d13df1_15,aldfly moudov
3,site_1_41e6fe6504a34bf6846938ba78d13df1_20,aldfly chswar
4,site_1_41e6fe6504a34bf6846938ba78d13df1_25,aldfly
5,site_1_cce64fffafed40f2b2f3d3413ec1c4c2_5,aldfly
6,site_1_cce64fffafed40f2b2f3d3413ec1c4c2_10,nocall
7,site_1_cce64fffafed40f2b2f3d3413ec1c4c2_15,aldfly
8,site_1_cce64fffafed40f2b2f3d3413ec1c4c2_20,nocall
9,site_1_cce64fffafed40f2b2f3d3413ec1c4c2_25,nocall


## EOF